In [251]:
import pandas as pd
import numpy as np
import re

In [252]:
jobs = pd.read_excel('DataSource_NZSeek/NZ_Admin_JOBS.xlsx', header=None, skiprows=1)

In [253]:
jobs.tail()

,0,1,2,3,4,5
2703,Key Account Manager,https://www.seek.co.nz/job/50490062?type=stand...,Hays Talent Solutions,location: AucklandAucklandarea: Auckland Centr...,"27d ago,at",classification: Administration & Office Suppor...
2704,Executive Assistant,https://www.seek.co.nz/job/50488000?type=stand...,one eighty recruitment,location: WellingtonWellingtonarea: Wellington...,"27d ago,at",Competitive hourly rate
2705,Temporary Office Roles,https://www.seek.co.nz/job/50524865?type=stand...,Asset Recruitment Ltd,location: WaikatoWaikatoarea: HamiltonHamilton...,"20d ago,at",Competitive hourly rates $$
2706,Temporary Office Roles,https://www.seek.co.nz/job/50477118?type=stand...,Asset Recruitment Ltd,location: WaikatoWaikatoarea: HamiltonHamilton...,"28d ago,at",Competitive hourly rates $$
2707,Executive Assistant,https://www.seek.co.nz/job/50496571?type=stand...,Alpha Recruitment - NZ,location: WellingtonWellington,"26d ago,at",classification: Administration & Office Suppor...


In [254]:
salary = pd.concat([jobs[3], jobs[5]], axis=1)

In [255]:
salary.columns = ['s1', 's2']

In [256]:
salary

,s1,s2
0,location: Bay of PlentyBay of Plentyarea: Taur...,classification: Administration & Office Suppor...
1,location: Bay of PlentyBay of Plentyarea: Taur...,classification: Administration & Office Suppor...
2,location: AucklandAuckland,classification: Administration & Office Suppor...
3,location: SouthlandSouthlandarea: Invercargill...,classification: Administration & Office Suppor...
4,location: CanterburyCanterburyarea: Christchur...,classification: Administration & Office Suppor...
...,...,...
2703,location: AucklandAucklandarea: Auckland Centr...,classification: Administration & Office Suppor...
2704,location: WellingtonWellingtonarea: Wellington...,Competitive hourly rate
2705,location: WaikatoWaikatoarea: HamiltonHamilton...,Competitive hourly rates $$
2706,location: WaikatoWaikatoarea: HamiltonHamilton...,Competitive hourly rates $$


In [257]:
salary['s2'] = salary['s2'].apply(lambda x: np.nan if x.startswith('classification') else x.strip())
salary['s1'] = salary['s1'].where(salary['s2'].notnull(), np.nan)

In [258]:
s = pd.DataFrame(salary['s2'])

In [285]:
s

,s2,low,high
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
2703,NaN,NaN,NaN
2704,Competitive hourly rate,NaN,NaN
2705,Competitive hourly rates $$,NaN,NaN
2706,Competitive hourly rates $$,NaN,NaN


In [259]:
s[s['s2'].notnull() & s['s2'].str.contains(' per ')]

,s2
12,$20 per hour
17,$20 - $24.99 per hour
23,$19 to $22 per hour
25,$20 - $24.99 per hour
36,$20 - $24.99 per hour
48,$25 per hour
58,$60 000 - $70 000 per annum
59,$20.00 per hour
66,$20 - $24.99 per hour
77,$20.00 per hour.


In [271]:
import re


def salary_extraction_with_per(x):
    if x != np.nan and ' per ' in str(x):
        x = x.strip()
        # numbers = re.findall(r'\s*\d+.?\d*', x)
        numbers = re.findall(r'\s*\d+(?:[,\s]\d+)*(?:.\d+)?', x)
        numbers = [re.sub(',', '', num) for num in numbers]
        numbers = [num.replace(' ', '') for num in numbers]
        print(numbers)
        if len(numbers) == 1 and (isinstance(numbers, int) or isinstance(numbers, float)):
            low, high = round(float(numbers[0]), 2), round(float(numbers[0]), 2)
        elif len(numbers) == 2:
            low = round(float(numbers[0]), 2)
            high = round(float(numbers[1]), 2)
        else:
            low, high = np.nan, np.nan
        if 'annum' in x:
            # 年薪转化为时薪
            low = low / 52 / 40
            high = high / 52 / 40

            low = round(low, 2)
            high = round(high, 2)
        return low, high
        # return pd.DataFrame({'low': [low], 'high': [high]})

    else:
        # return pd.DataFrame({'low': [np.nan], 'high': [np.nan]})
        return np.nan, np.nan

In [272]:
salary_extraction_with_per('$20 - $24.99 per hour')

['20', '24.99']


(20.0, 24.99)

In [273]:
salary_extraction_with_per('$60 000 - $70 000 per annum')

['60000', '70000']


(28.85, 33.65)

In [274]:
salary_extraction_with_per('$20.00 per hour.')

['20.00']


(nan, nan)

In [275]:
salary_extraction_with_per('$70-75k per annum')

['70-75']


(nan, nan)

In [265]:
salary_extraction_with_per(s['s2'][12])

['20']


(20.0, 20.0)

In [283]:
s['low'] = s['s2'].apply(lambda x: salary_extraction_with_per(x)[0])
s['high'] = s['s2'].apply(lambda x: salary_extraction_with_per(x)[1])

['20']
['20', '24.99']
['19', '22']
['20', '24.99']
['20', '24.99']
['25']
['60000', '70000']
['20.00']
['20', '24.99']
['20.00']
['20', '24.99']
['21']
['55000', '60000']
['20', '24.99']
['20', '24.99']
['20', '29.99']
['20', '29.99']
['20', '29.99']
['20', '24.99']
['25', '28']
['20', '24.99']
['22']
['75000']
['20', '24.99']
['50000']
['20', '25']
['20', '24.99']
['19', '22']
['35', '39.99']
['20', '24.99']
['20', '24.99']
['19', '23']
['21', '23']
['40', '45']
['18.50', '23.50']
['25', '30']
['25', '29.99']
['25', '29.99']
['20', '40']
['20', '30']
['70-75']
['20']
['20', '24.99']
['19', '22']
['20', '24.99']
['20', '24.99']
['25']
['60000', '70000']
['20.00']
['20', '24.99']
['20.00']
['20', '24.99']
['21']
['55000', '60000']
['20', '24.99']
['20', '24.99']
['20', '29.99']
['20', '29.99']
['20', '29.99']
['20', '24.99']
['25', '28']
['20', '24.99']
['22']
['75000']
['20', '24.99']
['50000']
['20', '25']
['20', '24.99']
['19', '22']
['35', '39.99']
['20', '24.99']
['20', '24.99']
[

In [284]:
s

,s2,low,high
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
2703,NaN,NaN,NaN
2704,Competitive hourly rate,NaN,NaN
2705,Competitive hourly rates $$,NaN,NaN
2706,Competitive hourly rates $$,NaN,NaN


In [292]:
s['low'].value_counts()

20.00    19
25.00     4
19.00     3
28.85     1
26.44     1
35.00     1
21.00     1
40.00     1
18.50     1
Name: low, dtype: int64

In [291]:
s['high'].value_counts()

24.99    13
29.99     5
22.00     2
23.00     2
30.00     2
33.65     1
28.85     1
28.00     1
25.00     1
39.99     1
45.00     1
23.50     1
40.00     1
Name: high, dtype: int64

In [293]:
s['low'] = s['low'].fillna(20.00)
s['high'] = s['high'].fillna(24.99)

In [294]:
s

,s2,low,high
0,NaN,20.0,24.99
1,NaN,20.0,24.99
2,NaN,20.0,24.99
3,NaN,20.0,24.99
4,NaN,20.0,24.99
...,...,...,...
2703,NaN,20.0,24.99
2704,Competitive hourly rate,20.0,24.99
2705,Competitive hourly rates $$,20.0,24.99
2706,Competitive hourly rates $$,20.0,24.99
